In [3]:
import kgbench as kg

data = kg.load('mdgenre') # Load with numpy arrays, and train/validation split

loaded data mdgenre (54.48s).


In [4]:
print(data.num_classes)
print(data.datatypes())

12
['iri', 'http://kgbench.info/dt#base64Image', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#langString', 'http://www.w3.org/2001/XMLSchema#dateTime', 'http://www.w3.org/2001/XMLSchema#decimal', 'http://www.w3.org/2001/XMLSchema#string']


In [5]:
data.withheld[:15]

array([[263997,      1],
       [139883,      9],
       [220756,     11],
       [233568,      2],
       [141778,      5],
       [218163,      2],
       [147763,      4],
       [149772,     11],
       [155895,      2],
       [184777,      2],
       [118631,      2],
       [218571,      6],
       [255150,      2],
       [219682,     11],
       [155134,     11]], dtype=int32)

In [6]:
def create_uri_classes_lists(data, data_set):
    list_of_uris = []
    list_of_classes=[]
    for data_instance in data_set:
    #     print(data_instance)
        list_of_uris.append(data.i2e[data_instance[0]][0])
        list_of_classes.append(data_instance[1])
    return list_of_uris, list_of_classes

In [7]:
list_of_uris_train, list_of_classes_train = create_uri_classes_lists(data, data.training)

In [8]:
list_of_uris, list_of_classes = create_uri_classes_lists(data, data.withheld)

In [9]:
def show_lists(list1, list2):
    for i, (uri, y) in enumerate(zip(list1, list2)):
        print(uri, y)
        if i > 1:
            break
    print(len(list1), len(list2))

print("witheld:")
show_lists(list_of_uris, list_of_classes)
print("train:")
show_lists(list_of_uris_train, list_of_classes_train)


witheld:
http://www.wikidata.org/entity/Q86081603 1
http://www.wikidata.org/entity/Q1894071 9
http://www.wikidata.org/entity/Q5352076 11
500 500
train:
http://www.wikidata.org/entity/Q602283 2
http://www.wikidata.org/entity/Q987943 2
http://www.wikidata.org/entity/Q18198101 11
4863 4863


In [10]:
data.i2e[127365]

('http://www.wikidata.org/entity/Q16424132', 'iri')

In [1]:
import time

In [2]:
import rdflib
start_time = time.time()
g = rdflib.Graph()
g.parse("data/md_raw_without_Images-.nt")
print("--- %s seconds ---" % (time.time() - start_time))

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x7fe9da1cc940>
Traceback (most recent call last):
  File "/Users/roderickvanderweerdt/opt/anaconda3/envs/pykeen_env/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/Users/roderickvanderweerdt/opt/anaconda3/envs/pykeen_env/lib/python3.9/site-packages/isodate/isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: '-0800-01-01T00:00:00Z'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x7fe9da1cc940>
Traceback (most recent call last):
  File "/Users/roderickvanderweerdt/opt/anaconda3/envs/pykeen_env/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _c

--- 44.03610324859619 seconds ---


In [11]:
q_duration = """
SELECT DISTINCT ?movie_uri ?val
WHERE {
    ?movie_uri <http://www.wikidata.org/prop/direct/P2047> ?val .
}
"""

q_props = """
SELECT DISTINCT ?z
WHERE {
  ?x a ?z .
}
"""

def run_query(graph, query):
    qres = graph.query(query)
    for row in qres:
        print(row)
        break
        
run_query(g, q_duration)

(rdflib.term.URIRef('http://www.wikidata.org/entity/Q2549028'), rdflib.term.Literal('20', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal')))


In [12]:
q = """
SELECT DISTINCT ?entity_uri ?val
WHERE {{
    ?entity_uri {} ?val .
}}
"""


#monument_values = {year: <int>, population: <int>, codeNationalMonument: <int>}
#movie_values = {duration: <decimal>, boxoffice: <decimal>}
def add_entity_value(entity_val_dict, value_name, graph, query):
    qres = graph.query(query)
    print("query returned", len(qres), "results.")
    new_dict = 0
    existing_dict = 0
    doubles = 0
    for row in qres:
        entity_str = str(row.entity_uri)
        try:
            try:
#                 monument_val_dict[monument_str][value_name] = int((int(monument_val_dict[monument_str][value_name]) + int(row.val))/2)
                if entity_val_dict[entity_str][value_name] > float(row.val):
                    entity_val_dict[entity_str][value_name] = float(row.val)
#                 print(entity_val_dict[entity_str][value_name], row.val)
                doubles += 1
            except:
                entity_val_dict[entity_str][value_name] = float(row.val)
                existing_dict += 1
        except:
            new_dict += 1
            entity_val_dict[entity_str] = {}
            entity_val_dict[entity_str][value_name] = float(row.val)
    print("values for new entity:", new_dict, "\nvalues for previous entities:", existing_dict, "\nmultiple values for same entities:", doubles)
    print("entity_val_dict now contains values for", len(entity_val_dict), "entities.\n")
    return entity_val_dict

value_list = ["duration", "boxoffice", "cost"]
props_list = ["<http://www.wikidata.org/prop/direct/P2047>", "<http://www.wikidata.org/prop/direct/P2142>", "<http://www.wikidata.org/prop/direct/P2130>"]
entity_val_dict = {}
start_time = time.time()
for value_name, prop in zip(value_list, props_list):
    entity_val_dict = add_entity_value(entity_val_dict, value_name, g, q.format(prop))
print("--- %s seconds ---" % (time.time() - start_time))

query returned 8163 results.
values for new entity: 8099 
values for previous entities: 0 
multiple values for same entities: 64
entity_val_dict now contains values for 8099 entities.

query returned 844 results.
values for new entity: 2 
values for previous entities: 787 
multiple values for same entities: 55
entity_val_dict now contains values for 8101 entities.

query returned 698 results.
values for new entity: 0 
values for previous entities: 696 
multiple values for same entities: 2
entity_val_dict now contains values for 8101 entities.

--- 0.4757988452911377 seconds ---


In [13]:
def print_values(val_list, val_dict):
    values = ""
    for val in val_list:
        try:
            values += str(val_dict[val])+','
        except:
            values += ','
    return values[:-1] #remove last ','
        
def write_entities_md(list_of_entities, list_of_y, val_list, entity_val_dict, new_fn):
    un_found = [0,0]
    with open(new_fn, 'w') as f:
        f.write("entity,y,"+','.join(val_list)+'\n')
        for entity, y in zip(list_of_entities, list_of_y):
            try:
                f.write(entity+','+str(y)+','+print_values(val_list, entity_val_dict[entity])+'\n')
                un_found[0] += 1
            except:
                f.write(entity+','+str(y)+','+print_values(val_list, {})+'\n')
#                 print(entity+','+str(y)+','+print_values(val_list, {}))
#                 print("no values found for entity:", entity)
                un_found[1] += 1
    print("found values for", un_found[0], "entities")
    print("couldn't find values for", un_found[1], "entities")

In [14]:
write_entities_md(list_of_uris_train, 
                   list_of_classes_train, 
                   value_list, 
                   entity_val_dict, 
                   "data/entities_md_raw_TRAIN.csv")

write_entities_md(list_of_uris, 
                   list_of_classes, 
                   value_list, 
                   entity_val_dict,
                   "data/entities_md_raw_VALIDATION.csv")


found values for 4236 entities
couldn't find values for 627 entities
found values for 426 entities
couldn't find values for 74 entities
